In [37]:
from scapy.all import rdpcap
from scapy.layers.dot11 import Dot11Elt

In [38]:
capture = rdpcap("../../data/raw/TEST/iPhone11_C.pcap")

In [39]:
EXTENDED_CAP = {
    0: "20_40_BSS_Coexistence_Management_Support",
    1: "Reserved_1",
    2: "Extended_Channel_Switching",
    3: "Reserved_2",
    4: "PSMP_Capability",
    5: "Reserved_3",
    6: "SPSMP_Support",
    7: "Event",
    8: "Diagnostics",
    9: "Multicast_Diagnostic",
    10: "Location_Tracking",
    11: "FMS",
    12: "Proxy_Arp_Service",
    13: "Collocated_Interference_Reporting",
    14: "Civic_Location",
    15: "Geospatial_Location",
    16: "TFS",
    17: "WNM_Sleep_Mode",
    18: "TIM_Broadcast",
    19: "BSS_Transition",
    20: "QoS_Traffic_Capability",
    21: "AC_Station_Count",
    22: "Multiple_BSSID",
    23: "Timing_Measurement",
    24: "Channel_Usage",
    25: "SSID_List",
    26: "DMS",
    27: "UTC_Timing",
    28: "TPU_Buffer_STA_Support",
    29: "TDLS_Peer_PSM",
    30: "TDLS_Channel_Switching",
    31: "Interworking",
    32: "QoS_Map",
    33: "EBR",
    34: "SSPN_Interface",
    35: "Reserved_4",
    36: "MSGCF_Capability",
    37: "TDLS_Support",
    38: "TDLS_Prohibited",
    39: "TDLS_Channel_Switching_Prohibited",
    40: "Reject_Unadmitted_Frame",
    41: "Service_Interval_Granularity",
    42: "Service_Interval_Granularity",
    43: "Service_Interval_Granularity",
    44: "Identifier_Location",
    45: "U_APSD_Coexistence",
    46: "WNM_Notification",
    47: "QAB_Capability",
    48: "UTF_8_SSID",
    49: "QMF_Activated",
    50: "QMF_ReconfigurationActivated",
    51: "Robust_AV_Streaming",
    52: "Advanced_GCR",
    53: "Mesh_GCR",
    54: "SCS",
    55: "QLoad_Report",
    56: "Alternate_EDCA",
    57: "Unprotected_TXOP",
    58: "Protected_RXOP",
    59: "Reserved_5",
    60: "Protected_QLoad_Report",
    61: "TDLS_Wide_Bandwidth",
    62: "Operating_Mode_Notification",
    63: "Max_Number_Of_MSDUs_In_A_MPDU",
    64: "Max_Number_Of_MSDUs_In_A_MPDU",
    65: "Channel_Schedule_Management",
    66: "Geodatabase_Inband_Enabling_Signal",
    67: "Network_Channel_Control",
    68: "White_Space_Map",
    69: "Channel_Availability_Query",
    70: "FTM_Responder",
    71: "FTM_Initiator",
    72: "Reserved_6",
    73: "ESM_Capability",
    74: "Future_Channel_Guidance",
}

In [40]:
capture[0]

<RadioTap  version=0 pad=0 len=15 present=Flags+Rate+Channel+dBm_AntSignal Flags=FCS Rate=1.0 Mbps ChannelFrequency=2437 ChannelFlags=CCK+2GHz dBm_AntSignal=-66 dBm notdecoded='' |<Dot11FCS  subtype=Probe Request type=Management proto=0 FCfield= ID=0 addr1=ff:ff:ff:ff:ff:ff (RA=DA) addr2=9e:d1:4e:71:48:dd (TA=SA) addr3=ff:ff:ff:ff:ff:ff (BSSID/STA) SC=10096 fcs=0x3d6a4b87 |<Dot11ProbeReq  |<Dot11Elt  ID=SSID len=0 info='' |<Dot11EltRates  ID=Supported Rates len=4 rates=[1.0(B) Mbps, 2.0(B) Mbps, 5.5(B) Mbps, 11.0(B) Mbps] |<Dot11EltRates  ID=Extended Supported Rates len=8 rates=[6.0 Mbps, 9.0 Mbps, 12.0 Mbps, 18.0 Mbps, 24.0 Mbps, 36.0 Mbps, 48.0 Mbps, 54.0 Mbps] |<Dot11EltDSSSet  ID=DSSS Set len=1 channel=6 |<Dot11EltHTCapabilities  ID=HT Capabilities len=26 L_SIG_TXOP_Protection=0 Forty_Mhz_Intolerant=1 PSMP=0 DSSS_CCK=0 Max_A_MSDU=3839 o Delayed_BlockAck=0 Rx_STBC=0 Tx_STBC=0 Short_GI_40Mhz=0 Short_GI_20Mhz=1 Green_Field=0 SM_Power_Save=disabled Supported_Channel_Width=20Mhz LDPC_Co

In [41]:
packet = capture[0].getlayer(Dot11Elt, ID=127).info.hex()

In [42]:
packet

'0000080400000040'

In [43]:
def hex_string_to_binary(hex_string):
    # Initialize the resulting binary string
    binary_string = ""

    # Iterate every two characters in the hexadecimal string
    for i in range(0, len(hex_string), 2):
        # Extract the pair of characters
        hex_pair = hex_string[i : i + 2]
        # Convert the pair to a byte (decimal) in little endian format
        byte = int(hex_pair, 16)
        # Reverse the byte
        byte = int("{:08b}".format(byte)[::-1], 2)
        # Convert the byte to a binary string of 8 bits and concatenate
        binary_string += f"{byte:08b}"

    return binary_string

In [44]:
bin_packet = hex_string_to_binary(packet)

In [45]:
bin_packet

'0000000000000000000100000010000000000000000000000000000000000010'

In [46]:
# bin_packet = "00000000000000000000100000000100000000000000000001000000"

In [47]:
def extract_fields_from_binary(extended_cap, binary_string):
    # Initialize a list to hold tuples of field names and their corresponding bits
    extracted_fields = []

    # Convert the binary string to a list for easier access by index, and reverse it
    binary_list = list(binary_string)

    # Initialize a set to keep track of fields already added
    added_fields = set()

    # Iterate through each bit index in the extended capabilities dictionary
    for bit_index in range(len(binary_list)):
        # Check if this bit index is in the dictionary (to handle binary strings longer than the dictionary)
        if bit_index in extended_cap:
            field_name = extended_cap[bit_index]
            # If the field has not been added yet, proceed to extract its bits
            if field_name not in added_fields:
                # Find all bit indexes for this field
                bit_indexes = [
                    index for index, name in extended_cap.items() if name == field_name
                ]
                # Extract bits for the current field
                field_bits = "".join(
                    [
                        binary_list[index] if index < len(binary_list) else "0"
                        for index in bit_indexes
                    ]
                )
                # Add the field and its bits to the list
                extracted_fields.append((field_name, field_bits))
                # Mark this field as added
                added_fields.add(field_name)

    return extracted_fields


# Example binary string
# Ensure the binary string matches the expected length
# binary_string = bin_packet[:max(EXTENDED_CAP.keys())+1].ljust(max(EXTENDED_CAP.keys())+1, '0')

In [48]:
# Call the function and print the result
extracted_fields = extract_fields_from_binary(EXTENDED_CAP, bin_packet)
for field, bits in extracted_fields:
    print(f"{field}: {bits}")

20_40_BSS_Coexistence_Management_Support: 0
Reserved_1: 0
Extended_Channel_Switching: 0
Reserved_2: 0
PSMP_Capability: 0
Reserved_3: 0
SPSMP_Support: 0
Event: 0
Diagnostics: 0
Multicast_Diagnostic: 0
Location_Tracking: 0
FMS: 0
Proxy_Arp_Service: 0
Collocated_Interference_Reporting: 0
Civic_Location: 0
Geospatial_Location: 0
TFS: 0
WNM_Sleep_Mode: 0
TIM_Broadcast: 0
BSS_Transition: 1
QoS_Traffic_Capability: 0
AC_Station_Count: 0
Multiple_BSSID: 0
Timing_Measurement: 0
Channel_Usage: 0
SSID_List: 0
DMS: 1
UTC_Timing: 0
TPU_Buffer_STA_Support: 0
TDLS_Peer_PSM: 0
TDLS_Channel_Switching: 0
Interworking: 0
QoS_Map: 0
EBR: 0
SSPN_Interface: 0
Reserved_4: 0
MSGCF_Capability: 0
TDLS_Support: 0
TDLS_Prohibited: 0
TDLS_Channel_Switching_Prohibited: 0
Reject_Unadmitted_Frame: 0
Service_Interval_Granularity: 000
Identifier_Location: 0
U_APSD_Coexistence: 0
WNM_Notification: 0
QAB_Capability: 0
UTF_8_SSID: 0
QMF_Activated: 0
QMF_ReconfigurationActivated: 0
Robust_AV_Streaming: 0
Advanced_GCR: 0
Mes